In [2]:

import numpy as np
import matplotlib.pyplot as plt
import ema

In [ ]:
directory = "data"
file = "knight_no_ref_000.tsv"

sensors = ["REF","UL","LL","JW","TT","TB","TD","TL","LC","UI","J","OS","MS","PL"]
subcolumns = ["ID","frame","state","q0","qx","qy","qz","x","y","z"] 
